In [6]:

from scipy.sparse.linalg import spsolve
import numpy as np
from scipy.sparse import lil_matrix

In [10]:
def solve_journal_bearing_pressure(
    viscosity: float,
    speed_rad: float,
    diameter: float,
    length: float,
    clearance: float,
    eps: float,
    Ntheta: int = 80,
    Nz: int = 40
):
    mu = viscosity
    omega = -speed_rad
    R = diameter / 2.0
    L = length
    c = clearance

    theta = np.linspace(0.0, 2.0 * np.pi, Ntheta)
    z = np.linspace(-L/2.0, L/2.0, Nz)
    dtheta = theta[1] - theta[0]
    dz = z[1] - z[0]

    Theta, Z = np.meshgrid(theta, z, indexing="ij")

    h = c * (1.0 + eps * np.cos(Theta))

    Kth = h**3
    Kzz = h**3
    dh_dtheta = -c * eps * np.sin(Theta)
    S = 6.0 * mu * R * omega * dh_dtheta

    def idx(i, j):
        return i * (Nz - 2) + (j - 1)

    N_unknowns = Ntheta * (Nz - 2)
    A = lil_matrix((N_unknowns, N_unknowns))
    b = np.zeros(N_unknowns)

    Kth_iphalf = np.zeros((Ntheta, Nz))
    for i in range(Ntheta):
        ip = (i + 1) % Ntheta
        Kth_iphalf[i, :] = 0.5 * (Kth[i, :] + Kth[ip, :])

    Kzz_jphalf = 0.5 * (Kzz[:, 1:] + Kzz[:, :-1])

    for i in range(Ntheta):
        for j in range(1, Nz - 1):
            row = idx(i, j)

            ip = (i + 1) % Ntheta
            im = (i - 1) % Ntheta

            Kth_e = Kth_iphalf[i, j]
            Kth_w = Kth_iphalf[im, j]
            Kzz_n = Kzz_jphalf[i, j]
            Kzz_s = Kzz_jphalf[i, j - 1]

            diag = (Kth_e + Kth_w) / (R**2 * dtheta**2) + \
                (Kzz_n + Kzz_s) / dz**2
            A[row, row] = diag

            A[row, idx(ip, j)] = -Kth_e / (R**2 * dtheta**2)
            A[row, idx(im, j)] = -Kth_w / (R**2 * dtheta**2)

            if j + 1 <= Nz - 2:
                A[row, idx(i, j + 1)] = -Kzz_n / dz**2
            if j - 1 >= 1:
                A[row, idx(i, j - 1)] = -Kzz_s / dz**2

            b[row] = S[i, j]

    p_inner = spsolve(A.tocsr(), b)

    p = np.zeros((Ntheta, Nz))
    for i in range(Ntheta):
        for j in range(1, Nz - 1):
            p[i, j] = p_inner[idx(i, j)]

    return p, Theta, Z, h

In [1]:
def compute_squeeze_number(mu, nu, pa, R, c):
    """
    Compute the squeeze number (σ) for hydrodynamic lubrication.

    Args:
        mu (float): Dynamic viscosity of the lubricant (Pa·s).
        nu (float): Normal velocity (m/s).
        pa (float): Ambient pressure (Pa).
        R (float): Radius of the journal (m).
        c (float): Radial clearance (m).

    Returns:
        float: The computed value of σ.
    """
    return (12 * mu * nu * (R / c)**2) / pa

In [2]:
def compute_lambda(mu, omega, pa, R, c):
    """
    Compute the dimensionless parameter Lambda (Λ) for hydrodynamic lubrication.

    Args:
        mu (float): Dynamic viscosity of the lubricant (Pa·s).
        omega (float): Angular velocity of the journal (rad/s).
        pa (float): Ambient pressure (Pa).
        R (float): Radius of the journal (m).
        c (float): Radial clearance (m).

    Returns:
        float: The computed value of Λ.
    """
    return (6 * mu * omega * (R / c)**2) / pa

In [ ]:
def compute_stiffness_and_damping(L, R, delta_pk, delta_pc, delta_x, delta_y):
    """
    Compute the stiffness and damping coefficients for a journal bearing.

    Args:
        L (float): Length of the journal bearing (m).
        R (float): Radius of the journal bearing (m).
        delta_pk (function): In-phase dynamic pressure perturbation as a function of theta and z.
        delta_pc (function): Out-of-phase dynamic pressure perturbation as a function of theta and z.
        delta_x (float): Perturbation in the x-direction (m).
        delta_y (float): Perturbation in the y-direction (m).

    Returns:
        dict: A dictionary containing stiffness and damping coefficients (k_xx, k_xy, k_yx, k_yy, c_xx, c_xy, c_yx, c_yy).
    """
    import numpy as np

    # Define integration limits
    theta_vals = np.linspace(0, 2 * np.pi, 100)  # Angular positions
    z_vals = np.linspace(-L / 2, L / 2, 100)  # Axial positions

    # Initialize coefficients
    k_xx = k_xy = k_yx = k_yy = 0.0
    c_xx = c_xy = c_yx = c_yy = 0.0

    # Perform numerical integration for stiffness coefficients
    for theta in theta_vals:
        for z in z_vals:
            k_xx += -delta_pk(theta, z) * np.cos(theta) * R * (2 * np.pi / len(theta_vals)) * (L / len(z_vals)) / delta_x
            k_yx += -delta_pk(theta, z) * np.sin(theta) * R * (2 * np.pi / len(theta_vals)) * (L / len(z_vals)) / delta_x
            k_yy += -delta_pk(theta, z) * np.sin(theta) * R * (2 * np.pi / len(theta_vals)) * (L / len(z_vals)) / delta_y

    # Perform numerical integration for damping coefficients
    for theta in theta_vals:
        for z in z_vals:
            c_xx += -delta_pc(theta, z) * np.cos(theta) * R * (2 * np.pi / len(theta_vals)) * (L / len(z_vals)) / delta_x
            c_yx += -delta_pc(theta, z) * np.sin(theta) * R * (2 * np.pi / len(theta_vals)) * (L / len(z_vals)) / delta_x
            c_yy += -delta_pc(theta, z) * np.sin(theta) * R * (2 * np.pi / len(theta_vals)) * (L / len(z_vals)) / delta_y

    # Return results as a dictionary
    return {
        "k_xx": k_xx,
        "k_xy": k_xy,
        "k_yx": k_yx,
        "k_yy": k_yy,
        "c_xx": c_xx,
        "c_xy": c_xy,
        "c_yx": c_yx,
        "c_yy": c_yy
    }

In [ ]:
import numpy as np

def calculate_delta_pk_and_delta_pc(epsilon, mu, omega, R, c, theta, z):
    """
    Calculate the in-phase (delta_pk) and out-of-phase (delta_pc) components of the dynamic pressure.

    Args:
        epsilon (float): Eccentricity ratio (e/c).
        mu (float): Dynamic viscosity of the lubricant (Pa·s).
        omega (float): Angular velocity of the journal (rad/s).
        R (float): Radius of the journal (m).
        c (float): Radial clearance (m).
        theta (float): Angular position (radians).
        z (float): Axial position (m).

    Returns:
        tuple: delta_pk (float), delta_pc (float)
    """
    h = c * (1 + epsilon * np.cos(theta))  # Gap function
    h3 = h**3

    # In-phase component of the dynamic pressure
    delta_pk = (6 * mu * omega * R / h3) * np.cos(theta)

    # Out-of-phase component of the dynamic pressure
    delta_pc = (6 * mu * omega * R / h3) * np.sin(theta)

    return delta_pk, delta_pc

def calculate_nu(frequency):
    """
    Calculate the oscillation or perturbation angular frequency (nu).

    Args:
        frequency (float): Frequency of oscillation (Hz).

    Returns:
        float: Angular frequency (rad/s).
    """
    return 2 * np.pi * frequency

In [ ]:
diameter = 0.1  # meters
length = 0.03  # meters
load = 525  # Newtons
clearance = 0.1e-3  # meters
viscosity = 0.1  # Pa.s
speed_rpm = 1500

solve_journal_bearing_pressure(viscosity, speed_rpm * 2 * np.pi / 60, diameter, length, clearance, eps, Ntheta, Nz)

SyntaxError: invalid syntax (3743197149.py, line 8)